In [1]:
import pandas as pd


INDIR = r'./ons-family-spending/'
filename = INDIR + 'A6_precleaned_lesstax.xlsx'
sheetname = 0

df = pd.read_excel(filename, sheetname, index_col=0, header=0)

#delete rows with NaN as index
df.dropna(axis='index', inplace=True)

#rename column
df.rename({'Unnamed: 1':'Name', 'Lowest':1, 'Second': 2, 'Third': 3, 'Fourth':4,
    'Fifth':5, 'Sixth':6, 'Seventh':7, 'Eighth': 8, 'Ninth':9, 'Highest': 10}, axis=1, inplace=True)
df.rename_axis('id', axis=0, inplace=True)

#make all strings numeric
df.replace(':', '0.00', inplace=True)
df.replace('..', '0.00', inplace=True)
df.replace('~', '0.00', inplace=True)

for col_idx in range(1, 12):
    df[df.columns[col_idx]] = df.iloc[:, col_idx].astype('string').str.extract(r'\[?(\d+\.?\d*)\]?', expand=False)
    df[df.columns[col_idx]] = df.iloc[:, col_idx].astype('float')

#separate indices that are x.x.x, x.x and x
df_totals = df[df.index.str.fullmatch('.{1,2}')]
df_subtotals = df[df.index.str.fullmatch('.{1,2}\..')]
df_items =  df[df.index.str.fullmatch('.{1,2}\..\..{1,2}')]

#breakout multi-level indexing
item_indexes = df_items.index.str.extract(r'(?P<group_id>.{1,2})\.(?P<sub_id>.)\.(?P<item_id>.{1,2})')
item_indexes = item_indexes.astype('int')
df_items.reset_index(inplace=True)
df_items = df_items.join(item_indexes)
df_items.set_index(['group_id', 'sub_id', 'item_id'], inplace=True)

subtotals_indexes = df_subtotals.index.str.extract(r'(?P<group_id>.{1,2})\.(?P<sub_id>.)')
subtotals_indexes = subtotals_indexes.astype('int')
df_subtotals.reset_index(inplace=True)
df_subtotals = df_subtotals.join(subtotals_indexes)
df_subtotals.set_index(['group_id', 'sub_id'], inplace=True)

#not all categories go to third tier of indexing, put missing subtotals into df_items...
df_subtotals_copy = df_subtotals.copy(deep=True)
df_subtotals_copy.reset_index(inplace=True)
df_subtotals_copy['item_id'] = 1
df_subtotals_copy.set_index(['group_id', 'sub_id', 'item_id'], inplace=True)
df_subtotals_copy = df_subtotals_copy[~df_subtotals_copy.index.isin(df_items.index)]
df_items = pd.concat([df_items, df_subtotals_copy])
df_items.sort_index(inplace=True)

#write to files to allow detailed checking.
def write_xlsx(filename, df_totals, df_subtotals, df_items):
    xlsx = pd.ExcelWriter(filename)
    df_totals.to_excel(xlsx, 'totals')
    df_subtotals.to_excel(xlsx, 'subtotals')
    df_items.to_excel(xlsx, 'items')
    xlsx.close()

write_xlsx(INDIR + 'A6.xlsx', df_totals, df_subtotals, df_items)
# df_subtotals.drop(columns='id', inplace=True)
# df_items.drop(columns='id', inplace=True)

In [2]:
number_in_household = [1.3,	1.5, 1.8, 2.1, 2.4, 2.5, 2.7, 2.8, 3.1, 3.2, 2.3]
equivalization_factor = [n ** 0.5 for n in number_in_household]

def equivalize(df):

    df_equiv = df.copy(deep=True)
    for ic, col in enumerate(df.select_dtypes('number')):
        
        df_equiv.loc[:, col] = df.loc[:, col]/equivalization_factor[ic]
    
    return df_equiv


df_totals_eq, df_subtotals_eq, df_items_eq = map(equivalize, 
                                    [df_totals, df_subtotals, df_items])
write_xlsx(INDIR + 'A6_eq.xlsx', df_totals_eq, df_subtotals_eq, df_items_eq)


In [2]:
# make percent versions (% of income)
def as_percent_of_income(df):
    
    colsums = df.sum(axis=0, numeric_only=True)

    df_pc = df.copy(deep=True)
    for col in colsums.index:
        df_pc.loc[:, col] = 100 * df.loc[:, col] / colsums[col]
    return df_pc

df_totals_pc, df_subtotals_pc, df_items_pc = map(as_percent_of_income, 
                                    [df_totals, df_subtotals, df_items])
write_xlsx(INDIR + 'A6_pc.xlsx', df_totals_pc, df_subtotals_pc, df_items_pc)

In [3]:
#normalize percent of income by average % for that item
def norm_pc(df):

    df_norm = df.copy(deep=True)
    for col in df.select_dtypes('number'):
        
        df_norm.loc[:, col] = 100 * df.loc[:, col]/df.loc[:, 'All']

    df_norm= df_norm.sort_values(by=[10])
    return df_norm

df_totals_pc_norm, df_subtotals_pc_norm, df_items_pc_norm = map(
    norm_pc, [df_totals_pc, df_subtotals_pc, df_items_pc])

write_xlsx(INDIR + 'A6_pc_norm.xlsx', df_totals_pc_norm, df_subtotals_pc_norm, df_items_pc_norm)

In [4]:
#find which centile spends the highest and lowest fraction of expenditure on each item
def maxmin_centiles(df):
    dfcopy = df.copy()
    dfcopy.drop('All', axis=1, inplace=True)
    maxcent = dfcopy.idxmax(axis=1, skipna=True, numeric_only=True)    
    mincent = dfcopy.idxmin(axis=1, skipna=True, numeric_only=True)
    maxmincent = pd.concat([df['Name'], mincent, maxcent], axis=1)
    maxmincent.rename({0:'min', 1:'max'}, axis=1, inplace=True)
    maxmincent = maxmincent.sort_values(by=['max', 'min'], ascending=[True, False])
    return maxmincent

df_totals_pc_maxmincent, df_subtotals_pc_maxmincent, df_items_pc_maxmincent = map(maxmin_centiles, 
    [df_totals_pc, df_subtotals_pc, df_items_pc])

write_xlsx(INDIR + 'A6_pc_maxmincent.xlsx', df_totals_pc_maxmincent, df_subtotals_pc_maxmincent, df_items_pc_maxmincent)

In [5]:
#make long form dataframes for altair plotting
meltfun = lambda df, value_name : df.melt(id_vars='Name', var_name='Income Centile', value_name=value_name)

df_totals_long, df_totals_pc_long, df_subtotals_long, df_subtotals_pc_long = map(meltfun,
    [df_totals, df_totals_pc, df_subtotals, df_subtotals_pc], 
    ['Expenditure', r'% Expenditure', 'Expenditure', r'% Expenditure']) 

df_totals_pc_norm_long, df_subtotals_pc_norm_long, df_items_pc_norm_long = map(meltfun, 
    [df_totals_pc_norm, df_subtotals_pc_norm, df_items_pc_norm],
    [r'% of mean proportion of expenditure']*3)


In [6]:
import altair as alt

alt.renderers.enable("mimetype")

def plot_bump_chart(df):
    
    lastcolname = df.columns.values[-1]
    
    thischart = alt.Chart(df).mark_line(point = True).encode(
        x = alt.X("Income Centile:O"),
        y="rank:O",
        color=alt.Color("Name:N", sort='y', scale=alt.Scale(scheme='Category20'))
    ).transform_window(
        rank="rank()",
        sort=[alt.SortField(lastcolname, order="descending")],
        groupby=["Income Centile"]
    )
    return thischart
    # .properties(
    #     title="Bump Chart for Stock Prices",
    #     width=600,
    #     height=150,
    # )

thischart = plot_bump_chart(df_totals_pc_long)
thischart

c:\Users\mr_cp\anaconda3\envs\data_ml\lib\site-packages\altair\utils\core.py:283: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
